<a href="https://colab.research.google.com/github/jianyangma/hugging-face/blob/main/DistilBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
import transformers
from datasets import load_dataset

# Load the wikitext-2-raw-v1 dataset
dataset = load_dataset('wikitext', name='wikitext-2-raw-v1', split='train')

# Get the 11th example (index 10) of the train split
example = dataset[10]

# Mask the 6th token
words = example['text'].split()
words[5] = f'{tokenizer.mask_token}'
masked_text = ' '.join(words)
print(example)


{'text': ' The game \'s battle system , the BliTZ system , is carried over directly from Valkyira Chronicles . During missions , players select each unit using a top @-@ down perspective of the battlefield map : once a character is selected , the player moves the character around the battlefield in third @-@ person . A character can only act once per @-@ turn , but characters can be granted multiple turns at the expense of other characters \' turns . Each character has a field and distance of movement limited by their Action Gauge . Up to nine characters can be assigned to a single mission . During gameplay , characters will call out if something happens to them , such as their health points ( HP ) getting low or being knocked out by enemy attacks . Each character has specific " Potentials " , skills unique to each character . They are divided into " Personal Potential " , which are innate skills that remain unaltered unless otherwise dictated by the story and can either help or impede

In [27]:
from torch import nn
from transformers import AutoModelForMaskedLM, AutoTokenizer

# Set tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-cased')
model = AutoModelForMaskedLM.from_pretrained("distilbert-base-cased")

# Tokenize
inputs = tokenizer(masked_text, return_tensors="pt")
# Save mask index
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]

# Get result from model
token_logits = model(**inputs).logits
mask_token_logits = token_logits[0, mask_token_index, :]

# See top 5 predictions
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()
for token in top_5_tokens:
  print(masked_text.replace(tokenizer.mask_token, tokenizer.decode([token])))

The game 's battle system , the BliTZ system , is carried over directly from Valkyira Chronicles . During missions , players select each unit using a top @-@ down perspective of the battlefield map : once a character is selected , the player moves the character around the battlefield in third @-@ person . A character can only act once per @-@ turn , but characters can be granted multiple turns at the expense of other characters ' turns . Each character has a field and distance of movement limited by their Action Gauge . Up to nine characters can be assigned to a single mission . During gameplay , characters will call out if something happens to them , such as their health points ( HP ) getting low or being knocked out by enemy attacks . Each character has specific " Potentials " , skills unique to each character . They are divided into " Personal Potential " , which are innate skills that remain unaltered unless otherwise dictated by the story and can either help or impede a character 